In [1]:
import pandas as pd
import numpy as np

In [2]:
### 심화문제

In [3]:
import seaborn as sns
iris = sns.load_dataset("iris")
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [4]:
iris.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [5]:
iris.groupby('species')[['sepal_length','sepal_width','petal_length','petal_width']].mean()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [6]:
iris.groupby('species')[['sepal_length','sepal_width','petal_length','petal_width']].std()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,0.352490,0.379064,0.173664,0.105386
versicolor,0.516171,0.313798,0.469911,0.197753
virginica,0.635880,0.322497,0.551895,0.274650


In [7]:
import seaborn as sns
titanic = sns.load_dataset("titanic")
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [8]:
titanic['category1'] = titanic.apply(lambda r: r.sex if r.age >=20 else 'child', axis=1)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,NaN,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


In [9]:
titanic['age'] = titanic['age'].fillna(titanic['age'].mean())
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.000000,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.000000,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,29.699118,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child
889,1,1,male,26.000000,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.000000,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


In [10]:
titanic['age'] = titanic['age'].fillna(titanic['age'].mean())
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.000000,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.000000,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,29.699118,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child
889,1,1,male,26.000000,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.000000,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


In [11]:
titanic.groupby(['sex','survived'])[['survived']].count()

survived
sex    survived          
female 0               81
       1              233
male   0              468
       1              109

In [12]:
titanic.groupby(['class','survived',])[['survived']].count()

survived
class  survived          
First  0               80
       1              136
Second 0               97
       1               87
Third  0              372
       1              119

In [13]:
titanic.groupby(['embark_town','survived',])[['survived']].count()

survived
embark_town survived          
Cherbourg   0               75
            1               93
Queenstown  0               47
            1               30
Southampton 0              427
            1              217

In [14]:
bins = [1, 20, 30, 50, 70, 100]
labels = ["미성년자", "청년", "중년", "장년", "노년"]
cats = pd.cut(titanic.age, bins, labels=labels)
cats

0        청년
1        중년
2        청년
3        중년
4        중년
       ... 
886      청년
887    미성년자
888      청년
889      청년
890      중년
Name: age, Length: 891, dtype: category
Categories (5, object): [미성년자 < 청년 < 중년 < 장년 < 노년]

In [15]:
titanic['age_cat'] = pd.DataFrame(titanic.age, columns=['age'])
titanic["age_cat"] = pd.cut(titanic.age, bins,labels=labels)
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1,age_cat
0,0,3,male,22.000000,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,male,청년
1,1,1,female,38.000000,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,female,중년
2,1,3,female,26.000000,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,female,청년
3,1,1,female,35.000000,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,female,중년
4,0,3,male,35.000000,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,male,중년
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,male,청년
887,1,1,female,19.000000,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,child,미성년자
888,0,3,female,29.699118,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,child,청년
889,1,1,male,26.000000,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,male,청년


In [16]:
# d
titanic.groupby(['age_cat'])[['survived']].count()

,survived
age_cat,
미성년자,165
청년,407
중년,241
장년,59
노년,5


In [17]:
bins = [1, 50, 100]
labels = ["미성년자", "중년", "노년"]
cats = pd.qcut(bins, 3, labels=labels)
cats

[미성년자, 중년, 노년]
Categories (3, object): [미성년자 < 중년 < 노년]

In [18]:
ti1 = pd.DataFrame(titanic.age, columns=['age'])
ti1['age_cat1'] = pd.qcut(ti1.age, 3, labels=labels)
ti1

,age,age_cat1
0,22.000000,미성년자
1,38.000000,노년
2,26.000000,중년
3,35.000000,노년
4,35.000000,노년
...,...,...
886,27.000000,중년
887,19.000000,미성년자
888,29.699118,중년
889,26.000000,중년


In [19]:
titanic['age_cat1'] = pd.DataFrame(titanic.age, columns=['age'])
titanic['age_cat1'] = pd.qcut(titanic.age, 3, labels=labels)
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1,age_cat,age_cat1
0,0,3,male,22.000000,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,male,청년,미성년자
1,1,1,female,38.000000,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,female,중년,노년
2,1,3,female,26.000000,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,female,청년,중년
3,1,1,female,35.000000,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,female,중년,노년
4,0,3,male,35.000000,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,male,중년,노년
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,male,청년,중년
887,1,1,female,19.000000,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,child,미성년자,미성년자
888,0,3,female,29.699118,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,child,청년,중년
889,1,1,male,26.000000,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,male,청년,중년


In [30]:
titanic.groupby(['sex','age_cat1',])[['survived']].count()

survived  sex
sex    age_cat1               
female 미성년자           122  122
       중년              97   97
       노년              95   95
male   미성년자           179  179
       중년             207  207
       노년             191  191

In [35]:
titanic.groupby(['age_cat1','sex'])[['sex','survived']].mean()

survived
age_cat1 sex             
미성년자     female  0.713115
         male    0.206704
중년       female  0.701031
         male    0.164251
노년       female  0.821053
         male    0.198953

In [36]:
import seaborn as sns
mile_per_gallon = sns.load_dataset("mpg")
mile_per_gallon.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


In [50]:
#3-a
mile_per_gallon['hp_per_cc'] = mile_per_gallon.displacement / mile_per_gallon.horsepower
mile_per_gallon.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,hp_per_cc
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu,2.361538
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320,2.121212
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite,2.120000
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst,2.026667
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino,2.157143


In [51]:
mile_per_gallon

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,hp_per_cc
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu,2.361538
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320,2.121212
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite,2.120000
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst,2.026667
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino,2.157143
...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford mustang gl,1.627907
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup,1.865385
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge rampage,1.607143
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger,1.518987


In [56]:
#3-c
mile_per_gallon.groupby(['cylinders'])[['mpg']].mean()

,mpg
cylinders,
3,20.550000
4,29.286765
5,27.366667
6,19.985714
8,14.963107


In [57]:
#3-d
mile_per_gallon.groupby(['origin'])[['hp_per_cc']].mean()

,hp_per_cc
origin,
europe,1.405597
japan,1.304120
usa,2.048289
